In [5]:
import json
from tqdm import tqdm
import string
import itertools
import pandas as pd
import time
import csv
import os
import psycopg2 as p2
from psycopg2 import sql

## CVE Feed manipulations

In [6]:
def get_non_empty_cpe_from_right(cpe23uri_to_search):
    '''
    Удаляем все пустые значения справа
    cpe:2.3:h:eq-3:homematic_ccu2:-:*:*:*:*:*:*:* -> cpe:2.3:h:eq-3:homematic_ccu2
    '''
# cpe23uri_to_search = 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:*:*:*:*:*:node.js:*:*'
    splitted_cpe = cpe23uri_to_search.lstrip("cpe:2.3:").split(":")
    # reassembled_cpe = ":".join(["cpe:2.3"] + splitted_cpe[:3])
    # print(splitted_cpe)
    for i in range(len(splitted_cpe)-1, 0, -1):
        # print(i)
        if splitted_cpe[i] != "*":
            minimal_non_empty_from_right_cpe = ":".join(["cpe:2.3"] + splitted_cpe[:i+1])
            break
    return minimal_non_empty_from_right_cpe

In [7]:
# memory-heavy operation
with open("./nvdcpematch-1.0.json") as f:
    file = f.read()
    cpe_json = json.loads(file)

In [8]:
minimal_non_empty_from_right_cpe = []
cpe_names = []
for match_cpe in tqdm(cpe_json['matches']):
    minimal_non_empty_from_right_cpe.append(get_non_empty_cpe_from_right(match_cpe['cpe23Uri']))
    if match_cpe['cpe_name']:
        store_cpe_names = []
        for cpe_name in match_cpe['cpe_name']:
            store_cpe_names.append(cpe_name['cpe23Uri'])
        cpe_names.append(store_cpe_names)
    else:
        cpe_names.append([match_cpe['cpe23Uri']])

# Создаем датафрейм с обрезанными cpe справа и соответсвующими cpe_names
# df_from_cpe_feed = pd.DataFrame({'minimal_cpe': minimal_non_empty_from_right_cpe
#                                  , 'cpe_names': cpe_names})

# Создаем словарь с обрезанными cpe справа и соответсвующими cpe_names
dict_from_cpe_feed = dict(zip(minimal_non_empty_from_right_cpe, cpe_names))


100%|██████████| 498312/498312 [00:03<00:00, 124743.43it/s]


In [76]:
# minimal_cpe_np = df_from_cpe_feed['minimal_cpe'].to_numpy(object)
# cpe_names_np = df_from_cpe_feed['cpe_names'].to_numpy(object)

In [77]:
%%time
x = 'cpe:2.3:a:postgresql:postgresql:-'
for i in range(len(minimal_cpe_np)):
    if minimal_cpe_np[i] == x:
        print(cpe_names_np[i])
        break

['cpe:2.3:a:postgresql:postgresql:-:*:*:*:*:*:*:*', 'cpe:2.3:a:postgresql:postgresql:-:*:*:*:*:debian:*:*', 'cpe:2.3:a:postgresql:postgresql:-:*:*:*:*:ubuntu:*:*']
CPU times: user 42.9 ms, sys: 156 µs, total: 43 ms
Wall time: 42.4 ms


In [91]:
dict_from_cpe_feed

{}

In [90]:
df_from_cpe_feed.query("minimal_cpe == 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:*:*:*:*:*:node.js'").cpe_names.tolist()

[['cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:-:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.1.0:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.2.0:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.2.1:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.2.2:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.0:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.1:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.2:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.3:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.4:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.5:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.6:*:*:*:*:node.js:*:*',
  'cpe:2.3:a:\\@thi.ng\\/egf_pro

In [89]:
%%time
t1 = time.time()
df_from_cpe_feed.query("minimal_cpe == 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:*:*:*:*:*:node.js'").cpe_names.tolist()
print(f"Elapsed inside search_in_cpe: {time.time() - t1}")

Elapsed inside search_in_cpe: 0.011810779571533203
CPU times: user 14.2 ms, sys: 124 µs, total: 14.4 ms
Wall time: 11.9 ms


In [82]:
t1 = time.time()
d[x]
print(f"Elapsed inside search_in_cpe: {time.time() - t1}")

Elapsed inside search_in_cpe: 3.8623809814453125e-05


In [43]:
def check_versions_cpe(cpe,
                         versionStartIncluding=False, versionEndIncluding=False,
                         versionStartExcluding=False, versionEndExcluding=False,
                         logfile=False):
    '''
    Сравниваем СРЕ из cpe json feed с версиями CPE из CVE, если есть ограничения на версии
    '''
    # print(cpe)
    # print(versionStartIncluding, versionEndIncluding, versionStartExcluding, versionEndExcluding)

    # Если нет условий для сравнения по версиям и минимально совпадает начало CPE до версии    # (а оно совпадает, если дошло до этой функции), то все CPE подходят
    
    if versionStartIncluding == False and versionEndIncluding == False and \
                         versionStartExcluding == False and versionEndExcluding == False:
        return True   

    if versionStartIncluding and versionStartExcluding:
        print("Both versionStarted are present. Weird!")
    if versionEndIncluding and versionEndExcluding:
        print("Both versionEnd are present. Weird!")
    
    # удаляем буквы из версии. Например, как в https://nvd.nist.gov/vuln/detail/CVE-2022-24844#match-9944199
    # ...2.5.0b:*:*:*:*:*:*:*
    letters_and_chars = string.ascii_letters + "-,(){}[]"
    cpe_version = cpe.lstrip("cpe:2.3").split(":")[3]
    # Если проверяем CPE с "-". Например, cpe:2.3:a:postgresql:postgresql:-:*:*:*:*:*:*:*
    # или с '*'. Например, cpe:2.3:a:amazon:log4jhotpatch:*:*:*:*:*:*:*:* (CVE-2022-0070)
    if cpe_version == "-" or cpe_version == "*":
        return True
    # print(cpe_version)
    
    # необычные последовательности в версии
    for chars in ['\(aala.3\)c0', '\(aalb.3\)c0', '\(aaly.3\)c0', '\(aaky.3\)c0', '\(aakz.3\)c0'
                  , '_\(929\)', '_\(1360\)', 'udraw']:
        if chars in cpe_version:
            cpe_version = cpe_version.replace(chars, '')
    # print(cpe_version)
    for char in cpe_version:
        if char in letters_and_chars:
            cpe_version = cpe_version.replace(char, '')

    
    if versionStartIncluding:
        for chars in ['-h', 'a', 'b', '\(aala.3\)c', '-']:
            # print(chars, vers)
            if chars in versionStartIncluding:
                versionStartIncluding = versionStartIncluding.replace(chars, ".")
    if versionStartExcluding:
        for chars in ['-h', 'a', 'b', '\(aala.3\)c', '-']:
            if chars in versionStartExcluding:
                versionStartExcluding = versionStartExcluding.replace(chars, ".")
    if versionEndIncluding:
        for chars in ['-h', 'a', 'b', '\(aala.3\)c', '-']:
            if chars in versionEndIncluding:
                versionEndIncluding = versionEndIncluding.replace(chars, ".")
    if versionEndExcluding:
        for chars in ['-h', 'a', 'b', '\(aala.3\)c', '-']:
            if chars in versionEndExcluding:
                versionEndExcluding = versionEndExcluding.replace(chars, ".")

    # print(versionEndExcluding)
    # add trailing zeros to cpe version
    cpe_version = cpe_version.rstrip(".").lstrip(".")
    
    
    

    while len(cpe_version.split(".")) < 7:
        cpe_version = cpe_version + ".0"
    # print(cpe_version)
    try:
        cpe_version = list(map(int, cpe_version.split(".")))
    except:
        print(f'Error while converting version to list of ints for CPE {cpe}', file=open('cve_cpe_config_dir/logs.txt', 'a'))
        # list(map(int, cpe_version.split(".")))
        return False
        
    if versionStartIncluding:
        versionStartIncluding = versionStartIncluding.rstrip(".")
        while len(versionStartIncluding.split(".")) < 7:
            versionStartIncluding = versionStartIncluding + ".0"
        try:
            versionStartIncluding = list(map(int, versionStartIncluding.split(".")))
        except:
            versionStartIncluding = False
            print(f'Error while converting versionStartIncluding {versionStartIncluding} value\
                  from CPE {cpe}. So versionStartIncluding was set to False', file=open('cve_cpe_config_dir/logs.txt', 'a'))    
    if versionEndIncluding:
        versionEndIncluding = versionEndIncluding.rstrip(".")
        while len(versionEndIncluding.split(".")) < 7:
            versionEndIncluding = versionEndIncluding + ".0"
        try:
            versionEndIncluding = list(map(int, versionEndIncluding.split(".")))
        except:
            versionEndIncluding = False
            print(f'Error while converting versionEndIncluding {versionEndIncluding} value\
                  from CPE {cpe}. So versionEndIncluding was set to False', file=open('cve_cpe_config_dir/logs.txt', 'a'))
    if versionStartExcluding:
        versionStartExcluding = versionStartExcluding.rstrip(".")
        while len(versionStartExcluding.split(".")) < 7:
            versionStartExcluding = versionStartExcluding + ".0"
        try:
            versionStartExcluding = list(map(int, versionStartExcluding.split(".")))
        except:
            versionStartExcluding = False
            print(f'Error while converting versionStartExcluding {versionStartExcluding} value\
                  from CPE {cpe}. So versionStartExcluding was set to False', file=open('cve_cpe_config_dir/logs.txt', 'a'))
    if versionEndExcluding:
        versionEndExcluding = versionEndExcluding.rstrip(".")
        while len(versionEndExcluding.split(".")) < 7:
            versionEndExcluding = versionEndExcluding + ".0"
    # print(versionEndExcluding)
        try:
            versionEndExcluding = list(map(int, versionEndExcluding.split(".")))
        except:
            versionEndExcluding = False
            print(f'Error while converting versionEndExcluding {versionEndExcluding} value\
                  from CPE {cpe}. So versionEndExcluding was set to False', file=open('logs.txt', 'a'))
    # return versionEndExcluding
        
    # print(cpe_version)
    # print(versionStartIncluding, versionEndIncluding, versionStartExcluding, versionEndExcluding)


    start_flag = 0
    end_flag = 0

    # Версия продукта дообогащается нулями до 7 знаков, чтобы охватить крайние случаи

    # strictly higher
    if versionStartExcluding:
        if (cpe_version[0] > versionStartExcluding[0]):
            start_flag = 1
        elif (cpe_version[0] == versionStartExcluding[0]):
            if (cpe_version[1] > versionStartExcluding[1]):
                start_flag = 1
            elif (cpe_version[1] == versionStartExcluding[1]):
                if (cpe_version[2] > versionStartExcluding[2]):
                    start_flag = 1
                elif (cpe_version[2] == versionStartExcluding[2]):
                    if (cpe_version[3] > versionStartExcluding[3]):
                        start_flag = 1
                    elif (cpe_version[3] == versionStartExcluding[3]):
                        if (cpe_version[4] > versionStartExcluding[4]):
                            start_flag = 1
                        elif (cpe_version[4] == versionStartExcluding[4]):
                            if (cpe_version[5] > versionStartExcluding[5]):
                                start_flag = 1
                            elif (cpe_version[5] == versionStartExcluding[5]):
                                if (cpe_version[6] > versionStartExcluding[6]):
                                    start_flag = 1
    if versionEndIncluding == False and versionEndExcluding == False:
        end_flag = 1
    # equal or higher
    if versionStartIncluding:
        if (cpe_version[0] > versionStartIncluding[0]):
            start_flag = 1
        elif (cpe_version[0] == versionStartIncluding[0]):
            if (cpe_version[1] > versionStartIncluding[1]):
                start_flag = 1
            elif (cpe_version[1] == versionStartIncluding[1]):
                if (cpe_version[2] > versionStartIncluding[2]):
                    start_flag = 1
                elif (cpe_version[2] == versionStartIncluding[2]):
                    if (cpe_version[3] > versionStartIncluding[3]):
                        start_flag = 1
                    elif (cpe_version[3] == versionStartIncluding[3]):
                        if (cpe_version[4] > versionStartIncluding[4]):
                            start_flag = 1
                        elif (cpe_version[4] == versionStartIncluding[4]):
                            if (cpe_version[5] > versionStartIncluding[5]):
                                start_flag = 1
                            elif (cpe_version[5] == versionStartIncluding[5]):
                                if (cpe_version[6] >= versionStartIncluding[6]):
                                    start_flag = 1
    if versionEndIncluding == False and versionEndExcluding == False:
        end_flag = 1

    if versionEndExcluding:
    # strictly lower
        if (cpe_version[0] < versionEndExcluding[0]):
            end_flag = 1
        elif (cpe_version[0] == versionEndExcluding[0]):
            if (cpe_version[1] < versionEndExcluding[1]):
                end_flag = 1
            elif (cpe_version[1] == versionEndExcluding[1]):
                if (cpe_version[2] < versionEndExcluding[2]):
                    end_flag = 1
                elif (cpe_version[2] == versionEndExcluding[2]):
                    if (cpe_version[3] < versionEndExcluding[3]):
                        end_flag = 1
                    elif (cpe_version[3] == versionEndExcluding[3]):
                        if (cpe_version[4] < versionEndExcluding[4]):
                            end_flag = 1
                        elif (cpe_version[4] == versionEndExcluding[4]):
                            if (cpe_version[5] < versionEndExcluding[5]):
                                end_flag = 1
                            elif (cpe_version[5] == versionEndExcluding[5]):
                                if (cpe_version[6] < versionEndExcluding[6]):
                                    end_flag = 1
                    # print(versionStartExcluding, versionStartIncluding)
        if versionStartIncluding == False and versionStartExcluding == False:
            start_flag = 1
    # equal or higher
    elif versionEndIncluding:
        if (cpe_version[0] < versionEndIncluding[0]):
            end_flag = 1
        elif (cpe_version[0] == versionEndIncluding[0]):
            if (cpe_version[1] < versionEndIncluding[1]):
                end_flag = 1
            elif (cpe_version[1] == versionEndIncluding[1]):
                if (cpe_version[2] <= versionEndIncluding[2]):
                    end_flag = 1
                elif (cpe_version[2] == versionEndIncluding[2]):
                    if (cpe_version[3] < versionEndIncluding[3]):
                        end_flag = 1
                    elif (cpe_version[3] == versionEndIncluding[3]):
                        if (cpe_version[4] < versionEndIncluding[4]):
                            end_flag = 1
                        elif (cpe_version[4] == versionEndIncluding[4]):
                            if (cpe_version[5] < versionEndIncluding[5]):
                                end_flag = 1
                            elif (cpe_version[5] == versionEndIncluding[5]):
                                if (cpe_version[6] <= versionEndIncluding[6]):
                                    end_flag = 1
                    # print(versionStartExcluding, versionStartIncluding)
        if versionStartIncluding == False and versionStartExcluding == False:
            start_flag = 1
    
    # print(versionStartIncluding, versionStartExcluding)
    # print(start_flag, end_flag)
    return start_flag==1 and end_flag==1



In [131]:
check_versions_cpe('cpe:2.3:a:webtoprint:web_to_print_shop\:udraw:*:*:*:*:*:wordpress:*:*'
                   , versionEndIncluding="6.6.8a")


Error while converting version to list of ints for CPE cpe:2.3:a:webtoprint:web_to_print_shop\:udraw:*:*:*:*:*:wordpress:*:*


False

In [39]:
def search_in_cpe_feed(cpe23uri_to_search: str,
                       versionStartIncluding=False, versionEndIncluding=False,
                        versionStartExcluding=False, versionEndExcluding=False):
    '''
    Получает CPE23Uri, обрезает до первой справа не пустой категории и ищет в CPE Feed
    '''
    # t1 = time.time()
    splitted_cpe = cpe23uri_to_search.lstrip("cpe:2.3:").split(":")
    # reassembled_cpe = ":".join(["cpe:2.3"] + splitted_cpe[:3])
    for i in range(len(splitted_cpe)-1, 0, -1):
        # print(i)
        if splitted_cpe[i] != "*":
            minimal_non_empty_from_right_cpe = ":".join(["cpe:2.3"] + splitted_cpe[:i+1])
            break
        # print(i, cpe23uri_to_search[i])
    # print(splitted_cpe)
    # print(minimal_non_empty_from_right_cpe)
    found_cpes = []

    # # !!!!!!!!!!!!!!!!!! slow !!!!!!!!!!!!!!!!!!
    # for cpe in cpe_json['matches']:
    #     if minimal_non_empty_from_right_cpe in 
    # cpe['cpe23Uri']:
    # # !!!!!!!!!!!!!!!!!! slow !!!!!!!!!!!!!!!!!!
    # available_cpes = df_from_cpe_feed[df_from_cpe_feed.minimal_cpe == minimal_non_empty_from_right_cpe].cpe_names.tolist()[0]
    try:
        available_cpes = dict_from_cpe_feed[minimal_non_empty_from_right_cpe]
    except:
        return [[], [], []]
    for cpe in available_cpes:
        # print(cpe)
        if cpe not in found_cpes \
            and check_versions_cpe(cpe, versionStartIncluding, 
                                    versionEndIncluding, versionStartExcluding, versionEndExcluding):
            found_cpes.append(cpe)            
            # found_cpes.append(cpe)
            # if len(cpe['cpe_name']) > 0:
            #     for cpe_name in cpe['cpe_name']:
            #         # print(cpe_name['cpe23Uri'])
            #         if cpe_name['cpe23Uri'] not in found_cpes \
            #             and check_versions_cpe(cpe_name['cpe23Uri'], versionStartIncluding, 
            #                                    versionEndIncluding, versionStartExcluding, versionEndExcluding):
            #             found_cpes.append(cpe_name['cpe23Uri'])
            # else:
            #     found_cpes.append(reassembled_cpe)
        else:
            "No CPES found in CPE json feed. Weird!" 
    # print(f"Elapsed inside search_in_cpe: {time.time() - t1}")
    return found_cpes

In [41]:
search_in_cpe_feed('cpe:2.3:a:bayashi:dopvcomet\\*:0009') == [[], [], []]

True

In [9]:
ex = 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:*:*:*:*:*:node.js'
search_in_cpe_feed(ex
                #    , versionStartIncluding="5.0"
                   , versionEndExcluding="2.5.1"
                   )

Elapsed inside search_in_cpe: 0.00015616416931152344


['cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:-:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.1.0:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.2.0:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.2.1:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.2.2:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.0:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.1:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.2:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.3:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.4:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.5:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.ng\\/egf:0.3.6:*:*:*:*:node.js:*:*',
 'cpe:2.3:a:\\@thi.ng\\/egf_project:\\@thi.n

In [52]:
def parse_config(config: json, old_counter: int):
    cve_name = config['cve']['CVE_data_meta']['ID']
    config_nodes = config['configurations']['nodes']
    len_nodes = len(config_nodes)

    # print(len_nodes)

    counter = old_counter
    cve_cpe_match = []
    for i in range(len_nodes):
        # Basic Configuration
        if config_nodes[i]['operator'] == "OR":
            for cpe_match_i in config_nodes[i]['cpe_match']:
                # t1 = time.time()
                # print(cpe_match_i)
                # print(type(cpe_match_i))
                cpe23Uri = cpe_match_i['cpe23Uri']
                # check for conditions in versions
                # cpe_match_keys = config_nodes[i]['cpe_match'].keys()
                versionStartIncluding = cpe_match_i.get('versionStartIncluding', 0)
                versionEndIncluding = cpe_match_i.get('versionEndIncluding', 0)
                versionStartExcluding = cpe_match_i.get('versionStartExcluding', 0)
                versionEndExcluding = cpe_match_i.get('versionEndExcluding', 0)
                # if 'versionStartIncluding' in cpe_match_keys or 'versionEndIncluding' in cpe_match_keys:
                    # versionStartIncluding = config_nodes[i]['cpe_match']
                # t2 = time.time()
                matched_cpes_from_feed = search_in_cpe_feed(cpe23Uri, versionStartIncluding, 
                                               versionEndIncluding, versionStartExcluding, versionEndExcluding)
                # t3 = time.time()
                for idx, cpe_i in enumerate(matched_cpes_from_feed):
                    cve_cpe_match.append([cve_name, cpe_i, counter])
                    counter += 1
                
                # print(f"Elapsed inside search_in_cpe t4-t3: {time.time() - t3}")
                # print(f"Elapsed inside search_in_cpe t3-t2: {t3 - t2}")
                # print(f"Elapsed inside search_in_cpe t2-t1: {t2 - t1}")
                # print("*"*50)

        elif config_nodes[i]['operator'] == "AND":
            len_children_and_operator = len(config_nodes[i]['children'])
            # Advanced configuration
            if len_children_and_operator == 0:
                # print("That is advanced configuration")
                len_cpe_match = len(config_nodes[i]['cpe_match'])
                cpe_uri23_list = [[] for _ in range(len_cpe_match)]
                for cpe_match_i, cpe_match in enumerate(config_nodes[i]['cpe_match']):
                    versionStartIncluding = cpe_match.get('versionStartIncluding', 0)
                    versionEndIncluding = cpe_match.get('versionEndIncluding', 0)
                    versionStartExcluding = cpe_match.get('versionStartExcluding', 0)
                    versionEndExcluding = cpe_match.get('versionEndExcluding', 0)
                    cpe23Uri = cpe_match['cpe23Uri']
                    matched_cpes_from_feed = search_in_cpe_feed(cpe23Uri, versionStartIncluding, 
                                versionEndIncluding, versionStartExcluding, versionEndExcluding)
                    cpe_uri23_list[cpe_match_i].extend(matched_cpes_from_feed)
                # [[1, 2], [3, 4], [5, 6]] -> [(1, 3, 5), (1, 3, 6),...]
                all_combinations_cpe_uri23_list = itertools.product(*cpe_uri23_list)
                for combination in all_combinations_cpe_uri23_list:
                    for cpe_inside_combination in combination:
                        cve_cpe_match.append([cve_name, cpe_inside_combination, counter])
                    counter += 1
                

            #  Running On\With Configuration
            else:
                if len_children_and_operator > 2:
                    print("Length of Running On\With more than 2   ", config_nodes)
                
                # CPE c On\With vulnerable: True частью
                cpe_vuln_true_list = []
                # CPE c On\With vulnerable: False частью
                cpe_vuln_false_list = []

                for node_child_i, nodes_child in enumerate(config_nodes[i]['children']):
                    # Непоследний потомок внутри Running On\With конфига
                    if nodes_child['operator'] == "OR" and (node_child_i+1) != len_children_and_operator:
                        for cpe_match_i in nodes_child['cpe_match']:
                            versionStartIncluding = cpe_match_i.get('versionStartIncluding', 0)
                            versionEndIncluding = cpe_match_i.get('versionEndIncluding', 0)
                            versionStartExcluding = cpe_match_i.get('versionStartExcluding', 0)
                            versionEndExcluding = cpe_match_i.get('versionEndExcluding', 0)
                            cpe23Uri = cpe_match_i['cpe23Uri']
                            # if 'versionStartIncluding' in cpe_match_keys or 'versionEndIncluding' in cpe_match_keys:
                                # versionStartIncluding = config_nodes[i]['cpe_match']
                            matched_cpes_from_feed = search_in_cpe_feed(cpe23Uri, versionStartIncluding, 
                                                        versionEndIncluding, versionStartExcluding, versionEndExcluding)
                            cpe_vuln_true_list.extend(matched_cpes_from_feed)
                            # for idx, cpe_i in enumerate(matched_cpes_from_feed):
                            #     cve_cpe_match.append([cpe23Uri, cpe_i, counter])
                            #     counter += 1
                    # Последний потомок внутри Running On\With конфига зарезервирован
                    # за On\With vulnerable: False частью
                    elif nodes_child['operator'] == "OR" and (node_child_i+1) == len_children_and_operator:
                        for cpe_match_i in nodes_child['cpe_match']:
                            versionStartIncluding = cpe_match_i.get('versionStartIncluding', 0)
                            versionEndIncluding = cpe_match_i.get('versionEndIncluding', 0)
                            versionStartExcluding = cpe_match_i.get('versionStartExcluding', 0)
                            versionEndExcluding = cpe_match_i.get('versionEndExcluding', 0)
                            cpe23Uri = cpe_match_i['cpe23Uri']
                            matched_cpes_from_feed = search_in_cpe_feed(cpe23Uri, versionStartIncluding, 
                                                        versionEndIncluding, versionStartExcluding, versionEndExcluding)
                            cpe_vuln_false_list.extend(matched_cpes_from_feed)
                
                # print(f"True list: {cpe_vuln_true_list}\nFalse list:{cpe_vuln_false_list}")
                for true_elem in cpe_vuln_true_list:
                    for false_elem in cpe_vuln_false_list:
                        cve_cpe_match.append([cve_name, true_elem, counter])
                        cve_cpe_match.append([cve_name, false_elem, counter])
                        counter += 1
                
    # print(config_nodes[0])

    # return config
    return cve_cpe_match, counter

100%|██████████| 24569/24569 [00:05<00:00, 4393.30it/s]


In [53]:
root_path = "./cve"

# iterating over folders with cve feed json fiels
for dir_path, _, _ in os.walk(root_path):
    # print(dir_path)
    if dir_path not in (root_path, './cve/downloaded') and dir_path == "./cve/nvdcve-1.1-2016":
        cve_feed_name = os.listdir(dir_path)[0]
        print(cve_feed_name)
        cve_feed_path = os.path.join(dir_path, cve_feed_name)

        with open(f"{cve_feed_path}") as f:
            file = f.read()
            json_file = json.loads(file)

        cve_cpe_config = []
        counter = 0
        with open('cve_cpe_config_dir/logs.txt', 'w+') as log_file:
            print(f'Starting iteration for {cve_feed_name}', file=log_file)
            for i in tqdm(range(len(json_file['CVE_Items']))): 
                # getting list of list with all cve_cpe_config pairs
                try:
                    i_cve_cpe_config, new_counter = parse_config(json_file['CVE_Items'][i], counter)
                except ValueError:
                    print(i)
                    i_cve_cpe_config, new_counter = parse_config(json_file['CVE_Items'][i], counter)
                    break
                # print(counter, new_counter)
                cve_cpe_config.extend(i_cve_cpe_config)
                counter = new_counter
            
            for i in range(len(cve_cpe_config)):
                if cve_cpe_config[i] == [[], [], []]:
                    cve_cpe_config.pop(i)

            # writing list of lists to csv
            with open(f'cve_cpe_config_dir/{cve_feed_name.rstrip("json")}.csv', 'w+') as csv_file:
                print(f'Writing to csv file for {cve_feed_name}')
                writer = csv.writer(csv_file)
                writer.writerow(['cve', 'cpe', 'config_id'])
                writer.writerows(cve_cpe_config)
                print(f'Done  with csv file for {cve_feed_name}')
                print('*'*50)

nvdcve-1.1-2016.json


 49%|████▉     | 5162/10547 [00:01<00:00, 5885.86it/s]

Length of Running On\With more than 2    [{'operator': 'AND', 'children': [{'operator': 'OR', 'children': [], 'cpe_match': [{'vulnerable': False, 'cpe23Uri': 'cpe:2.3:o:linux:linux_kernel:*:*:*:*:*:*:*:*', 'cpe_name': []}]}, {'operator': 'OR', 'children': [], 'cpe_match': [{'vulnerable': False, 'cpe23Uri': 'cpe:2.3:a:ffmpeg:ffmpeg:0.10:*:*:*:*:*:*:*', 'cpe_name': []}]}, {'operator': 'OR', 'children': [], 'cpe_match': [{'vulnerable': True, 'cpe23Uri': 'cpe:2.3:a:mozilla:firefox:*:*:*:*:*:*:*:*', 'versionEndIncluding': '47.0.1', 'cpe_name': []}, {'vulnerable': True, 'cpe23Uri': 'cpe:2.3:a:mozilla:firefox_esr:45.1.0:*:*:*:*:*:*:*', 'cpe_name': []}, {'vulnerable': True, 'cpe23Uri': 'cpe:2.3:a:mozilla:firefox_esr:45.1.1:*:*:*:*:*:*:*', 'cpe_name': []}, {'vulnerable': True, 'cpe23Uri': 'cpe:2.3:a:mozilla:firefox_esr:45.2.0:*:*:*:*:*:*:*', 'cpe_name': []}, {'vulnerable': True, 'cpe23Uri': 'cpe:2.3:a:mozilla:firefox_esr:45.3.0:*:*:*:*:*:*:*', 'cpe_name': []}]}], 'cpe_match': []}]


100%|██████████| 10547/10547 [00:02<00:00, 4260.02it/s]


Writing to csv file for nvdcve-1.1-2016.json
Done  with csv file for nvdcve-1.1-2016.json
**************************************************


In [49]:
len(cve_cpe_config)

794536

### TO-DO from 22.02.24

Ограничиваемся только cpe из nvdcpematch-1.0.json

Не все конфигурации cpe из cve есть в cpe, пока ограничиваемся те cpe, что есть в cpe feed 

Добавить CVSS 2

Сначала без версий

In [17]:
cve_cpe_config_2022

[['CVE-2022-0001', 'cpe:2.3:h:intel:celeron_j4005:-:*:*:*:*:*:*:*', 0],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:celeron_n4100:-:*:*:*:*:*:*:*', 1],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:celeron_n4000:-:*:*:*:*:*:*:*', 2],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:celeron_j4105:-:*:*:*:*:*:*:*', 3],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:pentium_silver_j5005:-:*:*:*:*:*:*:*', 4],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:pentium_silver_n5000:-:*:*:*:*:*:*:*', 5],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:core_i3-10110u:-:*:*:*:*:*:*:*', 6],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:core_i3-1005g1:-:*:*:*:*:*:*:*', 7],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:core_i5-10210u:-:*:*:*:*:*:*:*', 8],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:core_i5-10310y:-:*:*:*:*:*:*:*', 9],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:core_i5-10210y:-:*:*:*:*:*:*:*', 10],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:core_i5-1035g4:-:*:*:*:*:*:*:*', 11],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:core_i5-1035g7:-:*:*:*:*:*:*:*', 12],
 ['CVE-2022-0001', 'cpe:2.3:h:intel:core

In [12]:
cve_cpe_config_2022[0]

['CVE-2022-0001', 'cpe:2.3:h:intel:celeron_j4005:-:*:*:*:*:*:*:*', 0]

In [1]:
with open("logs.txt", mode="w+") as f:
    print('Hello', file=f)

In [2]:
! cat logs.txt

Hello


In [3]:
dbname = "vulns_scanner"
user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'

conn = p2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cur = conn.cursor()
table_name = 'cves'

In [7]:
query = sql.SQL(f'select * from {table_name}')
cur.execute(query)

In [8]:
rows = cur.fetchall()
for row in rows:
    print(row)

(2, 'avc')
